<a href="https://colab.research.google.com/github/lsampras/competitive-data-science/blob/master/Programming%20assignment%2C%20week%203%3A%20Mean%20encodings/Programming_assignment_week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [0]:
!wget 'https://github.com/lsampras/competitive-data-science/raw/master/Programming%20assignment%2C%20week%203:%20Mean%20encodings/grader.py'

In [0]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader
from sklearn.model_selection import KFold

# Read data

In [0]:
!wget 'https://github.com/lsampras/competitive-data-science/raw/master/readonly/final_project_data/sales_train.csv.gz'
sales = pd.read_csv('sales_train.csv.gz')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [5]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [6]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


#### Method 2

In [7]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [0]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [0]:
# YOUR CODE GOES HERE
y = all_data['target'].values
skf = KFold(5)
new_data = all_data
for train_idx,val_idx in skf.split(all_data,y):
    x_tr,x_val =  all_data.iloc[train_idx], all_data.iloc[val_idx]
    item_id_target_mean = x_tr.groupby('item_id').target.mean()
    x_val['item_target_enc'] = x_val['item_id'].map(item_id_target_mean)
    new_data.iloc[val_idx] = x_val
    

new_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = new_data['item_target_enc'].values
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [34]:
# YOUR CODE GOES HERE
target_sum = all_data.groupby('item_id')['target'].transform('sum')
target_count = all_data.groupby('item_id')['target'].transform('count')
all_data['item_target_enc'] = (target_sum - all_data.target.values) / (target_count - 1)
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.4803848311293002
Current answer for task Leave-one-out_scheme is: 0.4803848311293002


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [40]:
# YOUR CODE GOES HERE
globalmean = 0.3343
alpha = 100
nrows = all_data.groupby('item_id').target.count()
mean = all_data.groupby('item_id').target.mean()

#takes previous encoded feature value and regularizes it
encoded_feature = all_data['item_id'].map((mean*nrows + alpha*globalmean)/(alpha+nrows))



corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.4818198797097264
Current answer for task Smoothing_scheme is: 0.4818198797097264


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [14]:
encoded_feature

array([0.02272727, 0.05691057, 0.14141414, ..., 1.23869347, 0.29617021,
       1.08196721])

In [46]:
# YOUR CODE GOES HERE
# Calculate a mapping: {item_id: target_mean}
item_id_target_cumsum = all_data.groupby('item_id').target.cumsum() - all_data.target


item_id_target_cumcnt = all_data.groupby('item_id').target.cumcount()
all_data['item_target_enc'] = item_id_target_cumsum/item_id_target_cumcnt
all_data['item_target_enc'].fillna(0.3343, inplace = True)
encoded_feature = all_data['item_target_enc']

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.5025245211081697
Current answer for task Expanding_mean_scheme is: 0.5025245211081697


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [47]:
STUDENT_EMAIL = 'sampraslop995@gmail.com'
STUDENT_TOKEN = 'XiICwoX8H9yWYDOn'
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.4164590712798811
Task Leave-one-out_scheme: 0.4803848311293002
Task Smoothing_scheme: 0.4818198797097264
Task Expanding_mean_scheme: 0.5025245211081697


In [48]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
